In [1]:
import gzip
import math
import json
import numpy as np
import random
import pandas as pd
import sklearn
import string
from collections import defaultdict
from nltk.stem.porter import *
from sklearn.linear_model import LogisticRegression,Ridge,Lasso
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,mean_squared_error,confusion_matrix,classification_report,mean_absolute_error,f1_score,precision_score
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from sklearn.feature_selection import SelectKBest,chi2,mutual_info_classif
from sklearn.model_selection import train_test_split
import datetime as dt
#nltk.download('all')
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from gensim.models import Word2Vec
import dateutil
from scipy.sparse import lil_matrix # To build sparse feature matrices, if you like
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
from scipy import spatial

In [2]:
#Using a try-except block because it gives name error at some point of the file.
dataset = []

f = gzip.open("renttherunway_final_data.json.gz")
for l in f:
    try:
        d = eval(l)
        dataset.append(d)
    except:
        continue
        
f.close()

In [72]:
len(dataset)

192462

In [3]:
dataset[0]

{'fit': 'fit',
 'user_id': '420272',
 'bust size': '34d',
 'item_id': '2260466',
 'weight': '137lbs',
 'rating': '10',
 'rented for': 'vacation',
 'review_text': "An adorable romper! Belt and zipper were a little hard to navigate in a full day of wear/bathroom use, but that's to be expected. Wish it had pockets, but other than that-- absolutely perfect! I got a million compliments.",
 'body type': 'hourglass',
 'review_summary': 'So many compliments!',
 'category': 'romper',
 'height': '5\' 8"',
 'size': 14,
 'age': '28',
 'review_date': 'April 20, 2016'}

In [4]:
random.shuffle(dataset)
train=dataset[:len(dataset)-int(0.2*len(dataset))]
test=dataset[len(dataset)-int(0.2*len(dataset)):]

---------------------------------------------------- 
Pre Processing Data
------------------------------------------------

In [5]:
train_df=pd.DataFrame(train)
train_df

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,269467,34d,921642,115lbs,10,wedding,I ordered XS and S and ended up wearing the S ...,petite,It was perfect! Wish I could keep it. I receiv...,dress,"5' 2""",8,29,"September 12, 2017"
1,fit,200414,36dd,2654735,150lbs,10,everyday,This is a nice dressy jacket that is pretty an...,hourglass,Really pretty and flavoring,jacket,"5' 9""",12,40,"January 16, 2017"
2,fit,246648,34b,1499974,137lbs,10,party,I wore this to see Britney Spears in Vegas. I...,pear,Fun flirty and Sexy LBD,dress,"5' 7""",8,40,"March 1, 2017"
3,fit,178651,32ddd/e,2428645,165lbs,10,everyday,I wore this a few times while I had it for unl...,pear,So Unique!,romper,"5' 11""",14,22,"November 22, 2017"
4,fit,962282,38d+,125564,190lbs,8,formal affair,Dress was flattering and fit as I expected. On...,full bust,Black Tie event,gown,"5' 8""",39,38,"January 14, 2015"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153965,fit,780641,NaN,371022,120lbs,10,party,Fit was near perfect! Little roomy in the bus...,NaN,"Received many, many compliments on the dress! ...",dress,"5' 5""",4,33,"March 14, 2017"
153966,small,130067,34c,319191,150lbs,10,party,The fit was good in my waist and hips. I have...,athletic,I loved this dress! It was perfect for my work...,dress,"5' 9""",16,30,"December 21, 2017"
153967,fit,145109,34b,152998,118lbs,8,work,"Not as fitted as I'd have liked it to be, but ...",petite,Pretty dress for a work reception,dress,"5' 6""",4,32,"July 2, 2015"
153968,fit,542423,32c,141688,129lbs,10,formal affair,This dress was a stunner at a springtime black...,hourglass,Classically glamorous,gown,"5' 4""",12,37,"May 29, 2014"


In [6]:
train_df.isna().sum()

fit                   0
user_id               0
bust size         14662
item_id               0
weight            23928
rating                0
rented for            8
review_text           0
body type         11684
review_summary        0
category              0
height              533
size                  0
age                 774
review_date           0
dtype: int64

In [6]:
le=LabelEncoder()
# Fit the bust size
bust_le=le.fit(train_df['bust size'])
mapping_bust_size=dict(zip(bust_le.classes_,bust_le.transform(bust_le.classes_)))

# Fit the body type
body_le=le.fit(train_df['body type'])
mapping_body_type=dict(zip(body_le.classes_,body_le.transform(body_le.classes_)))

# Fit the fit
fit_le=le.fit(train_df['fit'])
mapping_fit=dict(zip(fit_le.classes_,fit_le.transform(fit_le.classes_)))

height_le=le.fit(train_df['height'])
mapping_height=dict(zip(height_le.classes_,height_le.transform(height_le.classes_)))

weight_le=le.fit(train_df['weight'])
mapping_weight=dict(zip(weight_le.classes_,weight_le.transform(weight_le.classes_)))

In [7]:
#train_df['bust_size_mapping']=le.fit_transform(train_df['bust size'])
#train_df['body_type_mapping']=le.fit_transform(train_df['body type'])
#train_df['height_mapping']=le.fit_transform(train_df['height'])
train_df['weight_mapping']=le.fit_transform(train_df['weight'])

median_weight=list(mapping_weight.keys())[list(mapping_weight.values()).index(int(train_df['weight_mapping'].median()))]
median_height=list(mapping_height.keys())[list(mapping_height.values()).index(int(train_df['height_mapping'].median()))]
median_bust_size= list(mapping_bust_size.keys())[list(mapping_bust_size.values()).index(int(train_df['bust_size_mapping'].median()))]
median_body_type= list(mapping_body_type.keys())[list(mapping_body_type.values()).index(int(train_df['body_type_mapping'].median()))]

As variables are categorical we impute with the mode of the data but keeping the name to be median still 

In [8]:
median_weight=list(mapping_weight.keys())[list(mapping_weight.values()).index(int(train_df['weight_mapping'].median()))]
median_height=statistics.mode(train_df['height'])
median_bust_size= statistics.mode(train_df['bust size'])
median_body_type= statistics.mode(train_df['body type'])

In [9]:
print(f'Median bust size is {median_bust_size}')
print(f'Median body type is {median_body_type}')
print(f'Median height is {median_height}')
print(f'Median weight is {median_weight}')

Median bust size is 34b
Median body type is hourglass
Median height is 5' 4"
Median weight is 140lbs


As we are imputing the categorical columns we use median. Rented for has very few missing values hence these rows can be dropped

df.to_dict('records') - converts a df into a list a dictionaries, records is a parameter

In [10]:
#train_df.drop(['bust_size_mapping','body_type_mapping','height_mapping','weight_mapping'],axis=1,inplace=True)
train_df.drop(['weight_mapping'],axis=1,inplace=True)

In [11]:
train_df.dropna(subset=['rented for'],inplace=True)

In [12]:
train_df.isna().sum()

fit                   0
user_id               0
bust size         14733
item_id               0
weight            23972
rating                0
rented for            0
review_text           0
body type         11565
review_summary        0
category              0
height              533
size                  0
age                 749
review_date           0
dtype: int64

In [13]:
#Adjust column dtypes to numeric to look at the distribution
train_df['age']=train_df['age'].astype(float)
median_age=train_df['age'].median()
train_df['age'].fillna(median_age,inplace=True)
train_df['age']=train_df['age'].astype(int)

In [14]:
train_df['rating']=train_df['rating'].astype(int)

In [15]:
train_df['weight']=train_df['weight'].fillna(median_weight)
train_df['height']=train_df['height'].fillna(median_height)

In [16]:
train_df['body type']=train_df['body type'].fillna(median_body_type)
train_df['bust size']=train_df['bust size'].fillna(median_bust_size)

In [17]:
train_df.isna().sum()

fit               0
user_id           0
bust size         0
item_id           0
weight            0
rating            0
rented for        0
review_text       0
body type         0
review_summary    0
category          0
height            0
size              0
age               0
review_date       0
dtype: int64

In [18]:
train_df.dtypes

fit               object
user_id           object
bust size         object
item_id           object
weight            object
rating             int32
rented for        object
review_text       object
body type         object
review_summary    object
category          object
height            object
size               int64
age                int32
review_date       object
dtype: object

In [19]:
train_df.to_csv('no_missing_train.csv',encoding='utf-8',index=False, errors='replace')

In [20]:
pre_processed_train=train_df.to_dict('records')

Let us pre process the test data

In [21]:
test_df=pd.DataFrame(test)

In [22]:
test_df.isna().sum()

fit                  0
user_id              0
bust size         3661
item_id              0
weight            5985
rating               0
rented for           1
review_text          0
body type         3072
review_summary       0
category             0
height             144
size                 0
age                210
review_date          0
dtype: int64

In [23]:
#Adjust column dtypes to numeric to look at the distribution
test_df['age']=test_df['age'].astype(float)
test_df['age'].fillna(median_age,inplace=True)
test_df['age']=test_df['age'].astype(int)

In [24]:
test_df['rating']=test_df['rating'].astype(int)

test_df['weight']=test_df['weight'].fillna(median_weight)
test_df['height']=test_df['height'].fillna(median_height)

test_df['body type']=test_df['body type'].fillna(median_body_type)
test_df['bust size']=test_df['bust size'].fillna(median_bust_size)

In [25]:
test_df.dropna(subset=['rented for'],inplace=True)

In [26]:
test_df.isna().sum()

fit               0
user_id           0
bust size         0
item_id           0
weight            0
rating            0
rented for        0
review_text       0
body type         0
review_summary    0
category          0
height            0
size              0
age               0
review_date       0
dtype: int64

In [27]:
test_df.to_csv('no_missing_test.csv',encoding='utf-8',index=False, errors='replace')

In [28]:
pre_processed_test=test_df.to_dict('records')

Since we want to use certain features extracted from the available features like month, year, review polarity .... we further use pandas to obtain that which make it easy to extract the feature vectors.

In [29]:
further_processing=train_df.copy()

In [30]:
# Sentiment analysis to find the polarity of the review 
sid = SentimentIntensityAnalyzer()
further_processing['review_polarity_score']=further_processing['review_text'].apply(lambda x : sid.polarity_scores(x)['compound'])

In [31]:
# A function that calculates bmi for the given weight in lbs and height in feet Example input is (' 5'8" ','157lbs')
def get_bmi(h,w):
    height_list= list(map(int,re.findall(r'\d+',h)))
    height_inches=sum([height_list[i]*12 if i==0 else height_list[i] for i in range(len(height_list))])
    weight_lb=int(re.findall(r'\d+',w)[0])
    bmi= 703 * weight_lb/(height_inches)**2
    return bmi

In [32]:
further_processing['BMI']=further_processing.apply(lambda x : get_bmi(x.height,x.weight),axis=1)

In [33]:
further_processing['date_converted']=pd.to_datetime(further_processing['review_date'],format='%B %d, %Y')
further_processing['year']=further_processing['date_converted'].dt.year
further_processing['month']=further_processing['date_converted'].dt.month

In [34]:
further_processing.columns.tolist()

['fit',
 'user_id',
 'bust size',
 'item_id',
 'weight',
 'rating',
 'rented for',
 'review_text',
 'body type',
 'review_summary',
 'category',
 'height',
 'size',
 'age',
 'review_date',
 'review_polarity_score',
 'BMI',
 'date_converted',
 'year',
 'month']

In [35]:
drop_columns=['review_text','review_summary','review_date','date_converted']

In [36]:
further_processing.drop(drop_columns,inplace=True,axis=1)

In [37]:
further_processing.to_csv('processed_train.csv',index=False)

In [75]:
list(further_processing.columns)


['fit',
 'user_id',
 'bust size',
 'item_id',
 'weight',
 'rating',
 'rented for',
 'body type',
 'category',
 'height',
 'size',
 'age',
 'review_polarity_score',
 'BMI',
 'year',
 'month']

In [38]:
test_further_processing=test_df.copy()

In [39]:
test_further_processing['review_polarity_score']=test_further_processing['review_text'].apply(lambda x : sid.polarity_scores(x)['compound'])
test_further_processing['BMI']=test_further_processing.apply(lambda x : get_bmi(x.height,x.weight),axis=1)

test_further_processing['date_converted']=pd.to_datetime(test_further_processing['review_date'],format='%B %d, %Y')
test_further_processing['year']=test_further_processing['date_converted'].dt.year
test_further_processing['month']=test_further_processing['date_converted'].dt.month


In [40]:
test_further_processing.drop(drop_columns,inplace=True,axis=1)

In [41]:
test_further_processing.to_csv('processed_test.csv',index=False)

In [42]:
pre_processed_train[0]

{'fit': 'fit',
 'user_id': '269467',
 'bust size': '34d',
 'item_id': '921642',
 'weight': '115lbs',
 'rating': 10,
 'rented for': 'wedding',
 'review_text': 'I ordered XS and S and ended up wearing the S since the XS was too tight on my chest. ',
 'body type': 'petite',
 'review_summary': 'It was perfect! Wish I could keep it. I received so many compliments!',
 'category': 'dress',
 'height': '5\' 2"',
 'size': 8,
 'age': 29,
 'review_date': 'September 12, 2017'}

---------------------------------------------------------------------------------
Baseline Model 
--------------------------------------------------------------------------------------

Let us consider this a classification task where we try to predict the rating given by the user

In [47]:
def get_even_rating(val):
    if(val>10):
        return 10
    if (math.floor(val)%2 == 0):
        return int(val)
    else:
        return int(val+1)

In [43]:
allUsers=set()
allItems=set()
usersPerItem=defaultdict(list)
itemsPerUser=defaultdict(list)
userRatings=defaultdict(list)
itemRatings=defaultdict(list)

for d in pre_processed_train:
    allUsers.add(d['user_id'])
    allItems.add(d['item_id'])
    usersPerItem[d['item_id']].append(d['user_id'])
    itemsPerUser[d['user_id']].append(d['item_id'])
    userRatings[d['user_id']].append(d['rating'])
    itemRatings[d['item_id']].append(d['rating'])

In [44]:
userAverageRating=defaultdict(float)
itemAverageRating=defaultdict(float)

for u in userRatings:
    userAverageRating[u]=sum(userRatings[u])/len(userRatings[u])
for u in itemRatings:
    itemAverageRating[u]=sum(itemRatings[u])/len(itemRatings[u])

In [71]:
globalAverageRating=int(statistics.mean(train_df['rating'].tolist()))

print(f'global average rating is {globalAverageRating}')

global average rating is 9


In [54]:
user_item_features=defaultdict(list)
userBMI=defaultdict(float)
userFittings=defaultdict(list)
ratingDict=defaultdict(float)
for d in pre_processed_train:
    
    bmi=get_bmi(d['height'],d['weight'])
    userBMI[d['user_id']]=bmi
    ratingDict[((d['user_id'],d['item_id']))]=d['rating']
    userFittings[d['user_id']].append(d['fit'])
        
    user_item_features[(d['user_id'],d['item_id'])]=[bmi,mapping_bust_size[d['bust size']],d['size'],mapping_fit[d['fit']],d['age']]

In [48]:
def baseline(user_id, bust_size, body_type, weight,height ,item_id ,size,review,age):
    
    if(user_id in userAverageRating and item_id in itemAverageRating):
        return (userAverageRating[user_id]+itemAverageRating[item_id]) / 2
    
    elif (user_id in userAverageRating and item_id not in itemAverageRating):
        return userAverageRating[user_id]
    
    elif (user_id not in userAverageRating and item_id in itemAverageRating):
        return itemAverageRating[item_id]
    else:
        return globalAverageRating
    

In [50]:
actual_baseline=[]
predictions_baseline=[]
for d in pre_processed_test:
    actual_baseline.append(d['rating'])
    #print(f'actual rating is {actualRatings[-1]}')
    predictions_baseline.append(baseline(d['user_id'],d['bust size'],d['body type'],d['weight'],d['height'],d['item_id'],d['size'],d['review_text'],d['age']))
    #print(f'Predicted rating is {predictions_baseline[-1]}')

In [51]:
mse_baseline=mean_squared_error(actual_baseline,predictions_baseline)
print(f'Mean Squared error of the Baseline is {mse_baseline}')

mae_baseline=mean_absolute_error(actual_baseline,predictions_baseline)
print(f'Mean Absolute error of the Baseline is {mae_baseline}')

rmse_baseline=mean_squared_error(actual_baseline,predictions_baseline,squared=False)
print(f'Root Mean Squared error of the Baseline is {rmse_baseline}')

Mean Squared error of the Baseline is 2.0216804513901376
Mean Absolute error of the Baseline is 1.0952124401556804
Root Mean Squared error of the Baseline is 1.4218580981905815


In [53]:
pred_updated=[get_even_rating(y) for y in predictions_baseline ]

accuracy_baseline=accuracy_score(actual_baseline,pred_updated)
print(f'Accuracy of the Baseline is {accuracy_baseline}')

f1=f1_score(actual_baseline,pred_updated,average='weighted')
print(f'F1 score of the Baseline is {f1}')

Accuracy of the Baseline is 0.5765243823231405
F1 score of the Baseline is 0.5647476913029352


9.092367547625697

------------------------------------------------------------------------------------
Hueristic Cosine similarity
----------------------------------------------------------------------------------------

user_item_features=defaultdict(list)
userBMI=defaultdict(float)
userFittings=defaultdict(list)
for d in pre_processed_train:
    
    bmi=get_bmi(d['height'],d['weight'])
    userBMI[d['user_id']]=bmi
    
    userFittings[d['user_id']].append(d['fit'])
        
    user_item_features[(d['user_id'],d['item_id'])]=[bmi,mapping_bust_size[d['bust size']],d['size'],mapping_fit[d['fit']]]

In [55]:
def CosineSimilarity(vec1,vec2):
    
    # Let us use adjusted cosine similarity because features are of diffrent range and any particular value of a feature which is high skews the similarity
    # To eliminate this we use adjusted cosine similarity, also cosine is not sensitive to the magnitude of its vectors
    
    mean_both=sum(sum(np.asarray(vec1),np.asarray(vec1))) / (len(vec1)+len(vec2))
    
    #print(f'vec1 is {vec1} and vec2 is {vec2} and mean diff is {mean_both}')
    #adjusted_vec1=np.asarray(vec1)
    #adjusted_vec2=np.asarray(vec2)
    
    adjusted_vec1=np.asarray(vec1)-mean_both
    adjusted_vec2=np.asarray(vec2)-mean_both
    
    dot=np.dot(adjusted_vec1,adjusted_vec2)
    denom1=np.linalg.norm(adjusted_vec1)
    denom2=np.linalg.norm(adjusted_vec2)
    
    #print(dot/(denom1*denom2))
    
    return dot/(denom1*denom2)

In [56]:
def generateFeatures(user_id,item_id,bust_size, body_type, weight,height,size,compare,review,age):
    
    current_user_bmi=get_bmi(height,weight)
    
    if(bust_size in mapping_bust_size):
        mapped_bust_size=mapping_bust_size[bust_size]
    else:
        req=int(''.join(re.findall(r'\d+',bust_size)))
        for val in mapping_bust_size:
            if(int(''.join(re.findall(r'\d+',val)))==req):
                mapped_bust_size= mapping_bust_size[val]
                break  
    
    if((user_id,item_id) in user_item_features):
        return user_item_features[user_id,item_id]
    
    # If the user is not in the train data, we have to find the users whose BMI is similair to the bmi of the test user and find the fit from the set of similar BMI users
    elif(item_id in usersPerItem and user_id not in itemsPerUser):
        # Finding out the users whose bmi is +-2 of the current test user
        list_of_users=[u for (u,i) in compare if(current_user_bmi-2<=userBMI[u]<=current_user_bmi+2)]
        #print(list_of_users)
        similar_user_fittings=[]
        for u in list_of_users:
            for f in userFittings[u]:
                similar_user_fittings.append(f)
        
        #print(f'no. of similar users with the bmi is {len(list_of_users)} and users are {list_of_users}')
        
        # If there is no user within the specific bmi range we assume that it the product will fit the test user
        if(len(similar_user_fittings)==0):
            fit_type='fit'
        else:
            fit_type=statistics.mode(similar_user_fittings)
        
        return [current_user_bmi,mapped_bust_size,size,mapping_fit[fit_type],age]
        
    elif(item_id not in usersPerItem and user_id in itemsPerUser):
        return [current_user_bmi,mapped_bust_size,size,mapping_fit['fit'],age]
    
    else:
        return [current_user_bmi,mapped_bust_size,size,mapping_fit['fit'],age]
    

def generateFeatures(user_id,item_id,bust_size, body_type, weight,height,size,compare):
    
    current_user_bmi=get_bmi(height,weight)
    
    if(bust_size in mapping_bust_size):
        mapped_bust_size=mapping_bust_size[bust_size]
    else:
        req=int(''.join(re.findall(r'\d+',bust_size)))
        for val in mapping_bust_size:
            if(int(''.join(re.findall(r'\d+',val)))==req):
                mapped_bust_size= mapping_bust_size[val]
                break  
    
    if((user_id,item_id) in user_item_features):
        return user_item_features[user_id,item_id]
    
    # If the user is not in the train data, we have to find the users whose BMI is similair to the bmi of the test user and find the fit from the set of similar BMI users
    elif(item_id in usersPerItem and user_id not in itemsPerUser):
        # Finding out the users whose bmi is +-2 of the current test user
        list_of_users=[u for (u,i) in compare if(current_user_bmi-2<=userBMI[u]<=current_user_bmi+2)]
        #print(list_of_users)
        similar_user_fittings=[]
        for u in list_of_users:
            for f in userFittings[u]:
                similar_user_fittings.append(f)
        
        #print(f'no. of similar users with the bmi is {len(list_of_users)} and users are {list_of_users}')
        
        # If there is no user within the specific bmi range we assume that it the product will fit the test user
        if(len(similar_user_fittings)==0):
            fit_type='fit'
        else:
            fit_type=statistics.mode(similar_user_fittings)
        
        return [current_user_bmi,mapped_bust_size,size,mapping_fit[fit_type]]
        
    elif(item_id not in usersPerItem and user_id in itemsPerUser):
        return [current_user_bmi,mapped_bust_size,size,mapping_fit['fit']]
    
    else:
        return [current_user_bmi,mapped_bust_size,size,mapping_fit['fit']]
    

In [61]:
def predictRating(user_id, bust_size, body_type, weight,height ,item_id ,size,review,age):
    similarity=[]
    rating=[]
    # Under test (user_id,item_id)
    test_pair=(user_id,item_id)
    # Comapring the similarity with list of (u,item_id) pairs who consumed the speciifc item 'item_id'
    comparing_smilarity_with=[]
    
    # As we are trying to compare the items if user is not present in train set it doesnt make the difference but we need the 'fit' variable
    # But for any new item there is no user history who used the new item
    for i in usersPerItem:
        if i==item_id:
            for u in usersPerItem[i]:
                rating.append(ratingDict[(u,item_id)]-userAverageRating[u])
                comparing_smilarity_with.append((u,item_id))
            break
    
    #print(f'test feature is {test_pair}')
    #print(f'other features are {comparing_smilarity_with}')
    test_feature_vector=generateFeatures(user_id,item_id,bust_size, body_type, weight,height,size,comparing_smilarity_with,review,age)
    #print(f'test feature is {test_feature_vector}')
    #print(f'other features are {comparing_smilarity_with}')
    other_feature_vectors=[generateFeatures(u,i,bust_size, body_type, weight,height,size,comparing_smilarity_with,review,age) for u,i in comparing_smilarity_with]
    #print(other_feature_vectors)
    
    if(len(other_feature_vectors)==0):
        return int(globalAverageRating)
    
    for v in other_feature_vectors:
        similarity.append(CosineSimilarity(test_feature_vector,v))
    
    weighted_avg=[x*y for (x,y) in zip(similarity,rating)]    
    
    if(user_id in userAverageRating and item_id in itemAverageRating):
        return ((userAverageRating[user_id]+itemAverageRating[item_id]) / 2) + (sum(weighted_avg)/sum(similarity))
    
    elif (user_id in userAverageRating and item_id not in itemAverageRating):
        return userAverageRating[user_id]+ (sum(weighted_avg)/sum(similarity))
    
    elif (user_id not in userAverageRating and item_id in itemAverageRating):
        return itemAverageRating[item_id]+ (sum(weighted_avg)/sum(similarity))
    else:
        return globalAverageRating
    
    
    '''
    if(mean_score>=0.983):
        return 10
    
    elif 0.970<=mean_score<0.983:
        return 9
    
    elif 0.955<=mean_score<0.970:
        return 8
    
    elif 0.94<=mean_score<0.96:
        return 7

    else:
        return 6
    '''
    
    
    

def predictRating(user_id, bust_size, body_type, weight,height ,item_id ,size):
    similarity=[]
    # Under test (user_id,item_id)
    test_pair=(user_id,item_id)
    # Comapring the similarity with list of (u,item_id) pairs who consumed the speciifc item 'item_id'
    comparing_smilarity_with=[]
    
    # As we are trying to compare the items if user is not present in train set it doesnt make the difference but we need the 'fit' variable
    # But for any new item there is no user history who used the new item
    for i in usersPerItem:
        if i==item_id:
            for u in usersPerItem[i]:
                comparing_smilarity_with.append((u,item_id))
            break
    
    #print(f'test feature is {test_pair}')
    #print(f'other features are {comparing_smilarity_with}')
    test_feature_vector=generateFeatures(user_id,item_id,bust_size, body_type, weight,height,size,comparing_smilarity_with)
    #print(f'test feature is {test_feature_vector}')
    #print(f'other features are {comparing_smilarity_with}')
    other_feature_vectors=[generateFeatures(u,i,bust_size, body_type, weight,height,size,comparing_smilarity_with) for u,i in comparing_smilarity_with]
    #print(other_feature_vectors)
    
    if(len(other_feature_vectors)==0):
        return int(globalAverageRating)
    
    for v in other_feature_vectors:
        similarity.append(CosineSimilarity(test_feature_vector,v))
    
    mean_score=statistics.mean(similarity)
    print(f'mean score is {mean_score}')
    
    if(mean_score>=0.983):
        return 10
    
    elif 0.970<=mean_score<0.983:
        return 9
    
    elif 0.955<=mean_score<0.970:
        return 8
    
    elif 0.94<=mean_score<0.96:
        return 7

    else:
        return 6


In [51]:
predictRating('91752','42j','full bust','220lbs','5\' 3"','1303995',57,'vhjaduahacckac',35)

mean score is 0.979237879943874


9

In [62]:
actualRatings=[]
predictions=[]
for d in pre_processed_test:
    actualRatings.append(d['rating'])
    print(f'actual rating is {actualRatings[-1]}')
    predictions.append(predictRating(d['user_id'],d['bust size'],d['body type'],d['weight'],d['height'],d['item_id'],d['size'],d['review_text'],d['age']))
    print(f'Predicted rating is {predictions[-1]}')

actual rating is 10
Predicted rating is 8.265748844071343
actual rating is 10
Predicted rating is 8.345352897170368
actual rating is 10
Predicted rating is 9.3887071520168
actual rating is 8
Predicted rating is 9.274827461529375
actual rating is 10
Predicted rating is 9.141439993119038
actual rating is 10
Predicted rating is 8.20909602519229
actual rating is 8
Predicted rating is 9.152357814485004
actual rating is 10
Predicted rating is 9.023887061815792
actual rating is 10
Predicted rating is 9.47133073378842
actual rating is 8
Predicted rating is 9.575436541454007
actual rating is 10
Predicted rating is 8.903297107024459
actual rating is 10
Predicted rating is 8.965920931986593
actual rating is 6
Predicted rating is 8.758804806441981
actual rating is 10
Predicted rating is 9.629411162197131
actual rating is 8
Predicted rating is 8.443301773493726
actual rating is 10
Predicted rating is 8.65555871113821
actual rating is 10
Predicted rating is 9.198791222978278
actual rating is 8
Predi

Predicted rating is 9.262141805424823
actual rating is 10
Predicted rating is 10.067070270548331
actual rating is 8
Predicted rating is 9.707514111585107
actual rating is 8
Predicted rating is 8.691389462932365
actual rating is 10
Predicted rating is 9.204639832244697
actual rating is 10
Predicted rating is 8.792665959299812
actual rating is 10
Predicted rating is 9.76541712582904
actual rating is 10
Predicted rating is 8.989985233881939
actual rating is 10
Predicted rating is 9.133257701529226
actual rating is 10
Predicted rating is 9.154797332494693
actual rating is 10
Predicted rating is 9.15836424512766
actual rating is 8
Predicted rating is 8.840051955654825
actual rating is 10
Predicted rating is 8.884000104601387
actual rating is 10
Predicted rating is 9.135924597867241
actual rating is 8
Predicted rating is 8.569009533833793
actual rating is 8
Predicted rating is 9.116248827928437
actual rating is 10
Predicted rating is 8.594149452571774
actual rating is 10
Predicted rating is 

Predicted rating is 9.063880449166437
actual rating is 8
Predicted rating is 8.909975117620414
actual rating is 10
Predicted rating is 9.180197871541411
actual rating is 8
Predicted rating is 8.690552577898094
actual rating is 10
Predicted rating is 6.871232498696058
actual rating is 8
Predicted rating is 9.654414201065112
actual rating is 8
Predicted rating is 8.77377033011452
actual rating is 10
Predicted rating is 8.701075575538487
actual rating is 6
Predicted rating is 9.003393146263805
actual rating is 10
Predicted rating is 9.14032816450732
actual rating is 6
Predicted rating is 8.202084539901543
actual rating is 8
Predicted rating is 10.302699885295205
actual rating is 10
Predicted rating is 8.918225174165432
actual rating is 10
Predicted rating is 9.669297421613043
actual rating is 10
Predicted rating is 7.534720857264882
actual rating is 6
Predicted rating is 8.954254757428412
actual rating is 10
Predicted rating is 8.95002638822824
actual rating is 6
Predicted rating is 9.827

Predicted rating is 8.48693475242716
actual rating is 8
Predicted rating is 9.25447558044002
actual rating is 10
Predicted rating is 9.05087103234802
actual rating is 6
Predicted rating is 9.080754714463877
actual rating is 10
Predicted rating is 8.602968460376042
actual rating is 10
Predicted rating is 10.190166771537891
actual rating is 8
Predicted rating is 9.214401982225814
actual rating is 10
Predicted rating is 9.23888700738057
actual rating is 4
Predicted rating is 8.154964193542803
actual rating is 10
Predicted rating is 8.810009465233543
actual rating is 10
Predicted rating is 10.330772842291797
actual rating is 10
Predicted rating is 9.698317362026458
actual rating is 6
Predicted rating is 9.250593999507641
actual rating is 10
Predicted rating is 9.18976354037991
actual rating is 6
Predicted rating is 8.231086033671508
actual rating is 10
Predicted rating is 9.200103048859377
actual rating is 10
Predicted rating is 9.368851939839603
actual rating is 8
Predicted rating is 8.44

Predicted rating is 8.057771648851574
actual rating is 10
Predicted rating is 9.420388194688394
actual rating is 10
Predicted rating is 9.732527137226088
actual rating is 10
Predicted rating is 9.356619620795419
actual rating is 10
Predicted rating is 8.996227682444884
actual rating is 10
Predicted rating is 8.376410557761911
actual rating is 8
Predicted rating is 8.79872384397865
actual rating is 10
Predicted rating is 9.061450124814987
actual rating is 10
Predicted rating is 8.175403648225211
actual rating is 10
Predicted rating is 9.405189972775013
actual rating is 10
Predicted rating is 9.430233971567683
actual rating is 8
Predicted rating is 9.20785882018518
actual rating is 10
Predicted rating is 9.274811966124599
actual rating is 10
Predicted rating is 9.2755561911044
actual rating is 4
Predicted rating is 9.06479747682182
actual rating is 10
Predicted rating is 9.719141701195687
actual rating is 10
Predicted rating is 9.077578263238921
actual rating is 10
Predicted rating is 8.

...

Predicted rating is 9.75677389819392
actual rating is 10
Predicted rating is 8.917089961776785
actual rating is 10
Predicted rating is 8.359563639176237
actual rating is 8
Predicted rating is 9.46681382604624
actual rating is 10
Predicted rating is 9.050659198816007
actual rating is 8
Predicted rating is 9.013517366360688
actual rating is 10
Predicted rating is 8.643636566679472
actual rating is 10
Predicted rating is 9.479228449823264
actual rating is 8
Predicted rating is 7.312026618096161
actual rating is 8
Predicted rating is 9.18013577936159
actual rating is 10
Predicted rating is 9.89003291821545
actual rating is 6
Predicted rating is 8.871521386441012
actual rating is 10
Predicted rating is 9.573378767868894
actual rating is 8
Predicted rating is 8.31993969722777
actual rating is 10
Predicted rating is 9.582935122564761
actual rating is 10
Predicted rating is 10.128296652366833
actual rating is 6
Predicted rating is 9.262078314091779
actual rating is 10
Predicted rating is 9.787

Predicted rating is 9.798483492656828
actual rating is 10
Predicted rating is 10.030596772851078
actual rating is 10
Predicted rating is 9.036273589722175
actual rating is 10
Predicted rating is 9.158881480131633
actual rating is 8
Predicted rating is 8.663886359670983
actual rating is 10
Predicted rating is 9.438960195346265
actual rating is 10
Predicted rating is 9.669666638924403
actual rating is 10
Predicted rating is 9.820374660525495
actual rating is 8
Predicted rating is 9.172066583193397
actual rating is 6
Predicted rating is 9.338841232765654
actual rating is 10
Predicted rating is 9.45912025857035
actual rating is 10
Predicted rating is 10.004968968451436
actual rating is 8
Predicted rating is 8.274780935473347
actual rating is 8
Predicted rating is 8.904737299075848
actual rating is 10
Predicted rating is 9.638139269483823
actual rating is 10
Predicted rating is 9.542402483588457
actual rating is 10
Predicted rating is 8.746587906186122
actual rating is 10
Predicted rating i

Predicted rating is 9.385529119722005
actual rating is 10
Predicted rating is 8.127688369199262
actual rating is 10
Predicted rating is 9.882034382953343
actual rating is 10
Predicted rating is 9.58263067828528
actual rating is 4
Predicted rating is 8.17063749430838
actual rating is 8
Predicted rating is 9.240423635058535
actual rating is 6
Predicted rating is 9.222684914269015
actual rating is 8
Predicted rating is 9.8304390694425
actual rating is 8
Predicted rating is 9.46805241243052
actual rating is 10
Predicted rating is 9.467386683280738
actual rating is 2
Predicted rating is 8.500289450801072
actual rating is 10
Predicted rating is 8.415097112943611
actual rating is 8
Predicted rating is 9.115379271902395
actual rating is 10
Predicted rating is 8.76634329444816
actual rating is 8
Predicted rating is 9.895132015660343
actual rating is 10
Predicted rating is 9.271990894888983
actual rating is 10
Predicted rating is 10.0025413806938
actual rating is 10
Predicted rating is 10.401865

Predicted rating is 9.014959678525157
actual rating is 8
Predicted rating is 8.451056374346123
actual rating is 8
Predicted rating is 9.004230321286585
actual rating is 8
Predicted rating is 7.792268127083011
actual rating is 8
Predicted rating is 9.65766812502388
actual rating is 10
Predicted rating is 9.514574153162902
actual rating is 8
Predicted rating is 9.79668380072366
actual rating is 8
Predicted rating is 8.938397527687759
actual rating is 10
Predicted rating is 8.685600017119645
actual rating is 6
Predicted rating is 9.59493415118151
actual rating is 10
Predicted rating is 9.680639045935273
actual rating is 10
Predicted rating is 9.063667763621412
actual rating is 6
Predicted rating is 7.709707394631074
actual rating is 10
Predicted rating is 9.69279980470805
actual rating is 10
Predicted rating is 8.091881497718461
actual rating is 10
Predicted rating is 9.939763966358429
actual rating is 8
Predicted rating is 7.883581295177941
actual rating is 10
Predicted rating is 9.48803

Predicted rating is 9.135968650815165
actual rating is 10
Predicted rating is 9.517493929071618
actual rating is 10
Predicted rating is 9.029201929993386
actual rating is 10
Predicted rating is 9.15618007351398
actual rating is 10
Predicted rating is 8.05789443406838
actual rating is 10
Predicted rating is 9.511105114800582
actual rating is 8
Predicted rating is 9.462139723796483
actual rating is 10
Predicted rating is 9.470972816276383
actual rating is 10
Predicted rating is 8.466074625374588
actual rating is 10
Predicted rating is 8.993407473459134
actual rating is 10
Predicted rating is 9.696267147216691
actual rating is 8
Predicted rating is 9.4668466463172
actual rating is 10
Predicted rating is 9.979423701573598
actual rating is 6
Predicted rating is 9.275071597792257
actual rating is 10
Predicted rating is 8.998809553050831
actual rating is 10
Predicted rating is 9.42719164087636
actual rating is 8
Predicted rating is 9.607388518607028
actual rating is 8
Predicted rating is 9.57

actualRatings=[]
predictions=[]
for d in pre_processed_test:
    actualRatings.append(d['rating'])
    print(f'actual rating is {actualRatings[-1]}')
    predictions.append(predictRating(d['user_id'],d['bust size'],d['body type'],d['weight'],d['height'],d['item_id'],d['size']))
    print(f'Predicted rating is {predictions[-1]}')

In [63]:
mse=mean_squared_error(actualRatings,predictions)
print(f'Mean Squared error of the Baseline is {mse}')

Mean Squared error of the Baseline is 2.104283506842427


In [64]:
mae=mean_absolute_error(actualRatings,predictions)
print(f'Mean Absolute error of the Baseline is {mae}')

Mean Absolute error of the Baseline is 1.0941438206358383


In [65]:
rmse=mean_squared_error(actualRatings,predictions,squared=False)
print(f'Root Mean Squared error of the Baseline is {rmse}')

Root Mean Squared error of the Baseline is 1.4506148719913314


As we are trying to compare which method is best ie. Regression or Classification, We try to bring the ratings predicted into the range of actual claases [2,4,6,8,10] so that there is better ground to compare. 
Thus if the predicted value is odd then just make it to the closest higher even value so that accuracy score can be obtained easily.

In [66]:
pred_updated=[get_even_rating(y) for y in predictions ]

In [67]:
accuracy_baseline=accuracy_score(actualRatings,pred_updated)
print(f'Accuracy of the Baseline is {accuracy_baseline}')

Accuracy of the Baseline is 0.5668078252058922


In [68]:
f1=f1_score(actualRatings,pred_updated,average='weighted')
print(f'F1 score of the Baseline is {f1}')

F1 score of the Baseline is 0.5606952378026406


In [59]:
from collections import Counter
Counter(actualRatings)

Counter({10: 24633, 8: 10919, 6: 2152, 4: 585, 2: 201})

In [64]:
Counter(pred_updated)

Counter({10: 31792, 8: 5110, 6: 1588})

In [65]:
cnfm=confusion_matrix(actualRatings,pred_updated)
cnfm

array([[    0,     0,     7,    33,   161],
       [    0,     0,    24,    75,   486],
       [    0,     0,    88,   276,  1788],
       [    0,     0,   352,  1369,  9198],
       [    0,     0,  1117,  3357, 20159]], dtype=int64)

In [66]:
clf_report=classification_report(actualRatings,pred_updated)
print(clf_report)

              precision    recall  f1-score   support

           2       0.00      0.00      0.00       201
           4       0.00      0.00      0.00       585
           6       0.06      0.04      0.05      2152
           8       0.27      0.13      0.17     10919
          10       0.63      0.82      0.71     24633

    accuracy                           0.56     38490
   macro avg       0.19      0.20      0.19     38490
weighted avg       0.48      0.56      0.51     38490



C:\Users\pspra\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pspra\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\pspra\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


----------------------------------------------------------------------------------------
Model using Factorization machines
------------------------------------------------------------------------------------------

from pywFM import pylibfm

In [63]:
from fastFM import als
from sklearn.feature_extraction import DictVectorizer

ModuleNotFoundError: No module named 'fastFM'

In [ ]:
def prepare_data(data):
    data_dict = data[['user_id', 'bust_size', 'body_type', 'height', 'weight', 'age', 'size', 'item_id', 'polarity_score', 'year', 'month']].to_dict(orient='records')
    vec = DictVectorizer()
    X = vec.fit_transform(data_dict)
    y = data['rating'].values
    return X, y

X_train, y_train = prepare_data(train_data)
X_test, y_test = prepare_data(test_data)

In [ ]:
#Initialize the model
fm = als.FMRegression(n_iter=100, rank=10, l2_reg_w=0.1, l2_reg_V=0.5)
#Fit the model to the training data
fm.fit(X_train, y_train)

In [ ]:
#Make predictions on the test data
y_pred = fm.predict(X_test)

#Evaluate the model using Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

In [ ]:
from sklearn.model_selection import GridSearchCV
from surprise.model_selection import train_test_split,GridSearchCV
from surprise import Dataset, Reader, SVD, SVDpp
from surprise import accuracy
import surprise

In [ ]:
ans=further_processing.drop(['rating'],axis=1)
ans

In [ ]:
reader = Reader(rating_scale=(0, 20))
dataset = Dataset.load_from_df(further_processing, reader)
train_set, valid_set = train_test_split(dataset, test_size=0.2, random_state=40)